## Threads mit Dask zusammenstellen

In [74]:
import bz2
import json
import itertools as it
from pathlib import Path
import pandas as pd
import collections as co
import dask.bag as db
from toolz import map, pipe, compose, do, partition_all

In [2]:
comments = Path("/media/jakob/bigdata/datasets/reddit/2016/RC_2016-07.bz2")

In [104]:
b = db.read_text(str(comments)).map(json.loads)
df = b.to_dataframe()

In [3]:
# df.head()

In [8]:
def comments_stream(comments):
    with bz2.open(comments) as c:
        while True:
            line = c.readline().decode('utf8')
            if len(line) == 0:
                break
            comment = json.loads(line)
#             print(comment["author"])
            yield comment

In [58]:
data = pd.DataFrame(list(it.islice(comments_stream(comments),1000)))

In [62]:
def edge(comment):
    rtype, parent = comment["parent_id"].split("_")
    node = comment["id"]
    return int(node,base=36), int(parent, base=36), int(rtype[1:])

def csv_line(edge):
    return "{:d}, {:d}, {:d}\n".format(*edge)
to_csv = compose(csv_line, edge)

In [66]:
next(map(to_csv, comments_stream(comments)))


'28591598364, 28591597119, 1\n'

In [70]:
log = []
inc = lambda x: x + 1
inc = compose(inc, do(log.append))
inc(1)

TypeError: do() missing 1 required positional argument: 'x'

In [75]:
edges = comments.parent / Path(comments.name + ".edges")
# 
with edges.open('w') as edge_file:
    out = compose(edge_file.write, to_csv)
    stream = map(out, comments_stream(comments))
#     do(edge_file.write, stream )
    for idx in stream:
        pass

In [10]:
# data = data.set_index("parent_id")
data.head()

,author,author_flair_css_class,author_flair_text,body,controversiality,created_utc,distinguished,edited,gilded,id,link_id,parent_id,retrieved_on,score,stickied,subreddit,subreddit_id,ups
0,DatDrummerGuy,red spy,,\*snorts*,0,1467331200,None,False,0,d4up6ho,t3_4qorui,t1_d4up5j3,1471069983,1,False,tf2,t5_2qka0,1
1,3inthebrowning,None,None,Not in the Midwest where there's grain product...,0,1467331200,None,False,0,d4up6hp,t3_4qm0f4,t1_d4ufu2e,1471069983,5,False,guns,t5_2qhc8,5
2,Heyutl,None,None,Very interesting. \n\nI actually had this thou...,0,1467331200,None,False,0,d4up6hq,t3_4qomst,t3_4qomst,1471069983,1,False,science,t5_mouw,1
3,Emfx,shadowhc,@Emfx - EVE Online bruh,What's bugged about it? Haven't kept up on me...,0,1467331200,None,False,0,d4up6hr,t3_4qobtm,t1_d4ulzv6,1471069983,1,False,pathofexile,t5_2sf6m,1
4,TheAwesomeMort,None,None,Aw man... I feel you.,0,1467331200,None,False,0,d4up6hs,t3_4qoivr,t1_d4up3ko,1471069983,1,False,AskReddit,t5_2qh1i,1


In [81]:

split = data.parent_id.str.split('_', expand=True)
split.columns = ["type", "parent"]
split = pd.concat([data, split], axis=1)
# split = split.set_index('id')

In [82]:
toplevel = split[~(split.type == "t1")]

In [83]:
co.Counter(split.parent).most_common()

[('4qool0', 29),
 ('4qlnnl', 27),
 ('4qnq5v', 23),
 ('4qooqv', 21),
 ('4qociz', 17),
 ('4qle9u', 17),
 ('4qotbw', 14),
 ('4qo1sr', 12),
 ('4qnmfg', 12),
 ('4qotus', 12),
 ('4qnq7q', 9),
 ('4qn97z', 9),
 ('4qml7a', 8),
 ('4qo06k', 8),
 ('d4umn9h', 8),
 ('4qnuoj', 8),
 ('4qo3ia', 8),
 ('d4uld3y', 7),
 ('4qn1fn', 7),
 ('4qmav5', 7),
 ('4qoqw1', 7),
 ('4qotfe', 7),
 ('4qn6ok', 7),
 ('4qotc2', 7),
 ('4qo7k0', 7),
 ('4qotsw', 7),
 ('4qot8g', 6),
 ('4qot19', 6),
 ('4qn4p7', 6),
 ('4qos6q', 6),
 ('4qnnw4', 6),
 ('4qn5cy', 6),
 ('4qotow', 6),
 ('4qm1dl', 5),
 ('4qnxvf', 5),
 ('4qotcc', 5),
 ('4qoayt', 5),
 ('4qo4ai', 5),
 ('4qmx7r', 5),
 ('4qotdv', 5),
 ('4qokrn', 5),
 ('4qomkq', 5),
 ('d4u9fzy', 5),
 ('4qmapq', 5),
 ('4qorgn', 5),
 ('4qmsmd', 5),
 ('4qoty5', 5),
 ('4qnuoy', 4),
 ('4qnw1l', 4),
 ('4qn6j0', 4),
 ('4qooyi', 4),
 ('4qoi2g', 4),
 ('4qos07', 4),
 ('4qkhs3', 4),
 ('d4ugdn5', 4),
 ('4qot4v', 4),
 ('4qlyaw', 4),
 ('4qnm1m', 4),
 ('4qnmfr', 4),
 ('4qoneh', 4),
 ('d4up3cb', 4),
 ('4qoam0

In [95]:
split[["id", "parent", "type"]]

,id,parent,type
0,d4up6ho,d4up5j3,t1
1,d4up6hp,d4ufu2e,t1
2,d4up6hq,4qomst,t3
3,d4up6hr,d4ulzv6,t1
4,d4up6hs,d4up3ko,t1
5,d4up6ht,4qot8g,t3
6,d4up6hu,d4ufdbi,t1
7,d4up6hv,d4uk7fz,t1
8,d4up6hw,d4uoeq2,t1
9,d4up6hx,d4up1j5,t1
